# Grasshopper Markov Chain

This notebook simulates trajectories drawn from the Markov chain shown in Figure 12.3 of the textbook.

In [1]:
from collections import Counter
import numpy as np
from tabulate import tabulate

def jump(origin: int, rng):
    """Jump randomly from an origin state to some destination state following the transition model shown in Fig 12.3"""
    assert origin in {-4, -3, -2, -1, 0, 1, 2, 3, 4}, "Unknown state"

    # probs: left, stay, right
    c = rng.choice(3, p=[0.25, 0.5, 0.25])
    
    if -4 < origin < 4:        
        if c == 0:
            return origin -1
        elif c == 1:
            return origin
        else:
            return origin + 1
    elif origin == -4:  # at the leftmost state, the probs are interpreted as (stay, stay, right)
        if c == 0:
            return origin 
        elif c == 1:
            return origin
        else:
            return origin + 1
    else:  # at the rightmost state, the probs are interpreted as (left, stay, stay)
        if c == 0:
            return origin - 1
        elif c == 1:
            return origin
        else:
            return origin


def sample_trajectory(init, num_steps, rng):
    """
    From init, draw a random trajectory with num_steps.
    """
    trajectory = [init]
    for t in range(num_steps):
        trajectory.append(jump(trajectory[-1], rng=rng))
    return trajectory


def simulate_stationary_distribution(init=0, num_samples=10000, num_steps=10, rng=np.random.default_rng(42)):
    """
    Simulate num_samples trajectories, each with num_steps, all of them starting from init
    then compute per step an MC estimate of the probability distribution at that step.
    Over time, consecutive probability distributions should differ less and less. 
    To show that, we compute TVD for adjacent steps. 
        TVD(p, q) = 0.5 * sum_x |p(x)-q(x)|
    TVD quantifies a notion of distance between distributions.
    We return a probability table with columns for step, states, and TVD.
    """

    samples = np.array([sample_trajectory(init, num_steps, rng=rng) for _ in range(num_samples)])

    table = []
    q = None
    for t in range(num_steps):
        c = Counter(samples[:,t])
        z = sum(c.values())
        p = [c.get(x, 0)/z for x in [-4, -3, -2, -1, 0, 1, 2, 3, 4]]
        assert np.isclose(np.sum(p), 1.0), "Too far from 1.0"
        if q is None:
            table.append([f"{t}"] + p + [])
            q = p
        else:
            d = 0.5*np.abs(np.array(q) - np.array(p)).sum()
            q = p
            table.append([f"{t}"] + p + [d])            
    return table

In [2]:
table = simulate_stationary_distribution(num_samples=1000, num_steps=21)
print(tabulate(table, floatfmt=".4f", headers=['t', -4, -3, -2, -1, 0, 1, 2, 3, 4, 'TVD']))

  t      -4      -3      -2      -1       0       1       2       3       4     TVD
---  ------  ------  ------  ------  ------  ------  ------  ------  ------  ------
  0  0.0000  0.0000  0.0000  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000
  1  0.0000  0.0000  0.0000  0.2550  0.4920  0.2530  0.0000  0.0000  0.0000  0.5080
  2  0.0000  0.0000  0.0580  0.2620  0.3630  0.2580  0.0590  0.0000  0.0000  0.1290
  3  0.0000  0.0140  0.0980  0.2280  0.3140  0.2400  0.0940  0.0120  0.0000  0.1010
  4  0.0000  0.0340  0.1130  0.2140  0.2850  0.2110  0.0970  0.0430  0.0030  0.0720
  5  0.0120  0.0450  0.1230  0.2060  0.2310  0.2030  0.1130  0.0490  0.0180  0.0700
  6  0.0170  0.0520  0.1410  0.1800  0.1990  0.1970  0.1230  0.0630  0.0280  0.0640
  7  0.0290  0.0650  0.1280  0.1740  0.1970  0.1810  0.1200  0.0610  0.0450  0.0420
  8  0.0410  0.0610  0.1350  0.1600  0.2090  0.1690  0.0960  0.0900  0.0390  0.0600
  9  0.0380  0.0830  0.1260  0.1860  0.1540  0.1690  0.1190  0.0690  0.0560  0.0880
